# Chapter03 word2vec
이번 장에는 추론 기반 기법을 사용한다.  
추론 과정에서 신경망을 사용하고, 여기서 word2vec이 등장.  

## 3.1 추론 기반 기법과 신경망

### 3.1.1 통계 기반 기법의 문제점
어휘가 100만개면 100만*100만 의 행렬을 계산해야 한다. 여기에 SVD를 적용하는건 무리다.  
(SVD의 복잡도는 O(n**3))  
통계기반기법은 데이터를 한번에 처리한다(batch). 추론기반기법은 조금씩 쓴다(minibatch)  

### 3.1.2 추론 기반 기법 개요
추론: 주변 단어(맥락)이 주어졌을 때, 빈칸의 단어를 추측하는 작업.  
ex: "you [   ] goodbye and i say hello" 

### 3.1.3 신경망에서의 단어 처리
단어를 one hot vector로 변환. 
index가 단어 id와 같으면 1, 아니면 0.  
이런 방법으로 입력의 크기를 고정한다.  

In [1]:
import numpy as np

C = np.array([1, 0, 0, 0, 0, 0, 0]) #input
W = np.random.randn(7, 3)           #weight
h = np.matmul(C, W)                 #hidden node
print(h)

[ 0.47167304  0.96859856 -1.29090961]


C is one hot vector.  
thus, C*W means we extracted single row from W.

In [2]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([1, 0, 0, 0, 0, 0, 0])
W = np.random.randn(7,3)
layer = MatMul(W) #?
h = layer.forward(c)
print(h)

[ 0.56953879  0.03512242 -0.51395428]


## 3.2 simple word2vec
we are going to use CBOW(Continuous bag-od-words) for neural network

### 3.2.1 CBOW model's inference process
target word: blank words.  
context: words around target word  
we use multiple inputs. N words for context => N inputs  
hidden layer's input will be (h1 + h2)/N   
output layer's neuron are map of our words.  
W(in) is distributed expression of words

In [3]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

#sample context data
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

#initializing weights
W_in = np.random.randn(7,3)
W_out = np.random.randn(3, 7)

#계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

#순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[ 1.22177788e+00  1.04258115e-01  1.05141753e+00 -1.94352697e-03
  -1.96416889e+00  2.98756978e-01 -8.24743898e-01]]


### 3.2.2 CBOW's training
we use softmax and CEE to measure loss.  
(skip-gram is our model)

### 3.2.3 word2vec's weight and distributed expression
W_in has information of words.  (7 x 3)  
W_out also has information of words. in column (3 x 7)  
what do we use?  we could use both of them. simply get mean.  
we will use W_in. which is common